<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Objectives" data-toc-modified-id="Objectives-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Objectives</a></span></li><li><span><a href="#Accessing-scraped-raw-data-in-AWS-S3" data-toc-modified-id="Accessing-scraped-raw-data-in-AWS-S3-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Accessing scraped raw data in AWS S3</a></span></li><li><span><a href="#How-many-months-(raw-data)-need-processing?" data-toc-modified-id="How-many-months-(raw-data)-need-processing?-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>How many months (raw data) need processing?</a></span></li><li><span><a href="#Read-in-raw-data-from-S3" data-toc-modified-id="Read-in-raw-data-from-S3-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Read in raw data from S3</a></span><ul class="toc-item"><li><span><a href="#Non-immigrant-data" data-toc-modified-id="Non-immigrant-data-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>Non-immigrant data</a></span></li><li><span><a href="#Immigrant-visa-data" data-toc-modified-id="Immigrant-visa-data-4.2"><span class="toc-item-num">4.2&nbsp;&nbsp;</span>Immigrant visa data</a></span></li></ul></li><li><span><a href="#Cleaning" data-toc-modified-id="Cleaning-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Cleaning</a></span><ul class="toc-item"><li><span><a href="#A-list-of-raw-dataframes" data-toc-modified-id="A-list-of-raw-dataframes-5.1"><span class="toc-item-num">5.1&nbsp;&nbsp;</span>A list of raw dataframes</a></span></li><li><span><a href="#Concatenated-one-long-dataframe" data-toc-modified-id="Concatenated-one-long-dataframe-5.2"><span class="toc-item-num">5.2&nbsp;&nbsp;</span>Concatenated one long dataframe</a></span></li><li><span><a href="#Remove-non-data-rows" data-toc-modified-id="Remove-non-data-rows-5.3"><span class="toc-item-num">5.3&nbsp;&nbsp;</span>Remove non-data rows</a></span><ul class="toc-item"><li><span><a href="#NIV" data-toc-modified-id="NIV-5.3.1"><span class="toc-item-num">5.3.1&nbsp;&nbsp;</span>NIV</a></span></li><li><span><a href="#IV" data-toc-modified-id="IV-5.3.2"><span class="toc-item-num">5.3.2&nbsp;&nbsp;</span>IV</a></span></li></ul></li><li><span><a href="#Split-all-in-one-column" data-toc-modified-id="Split-all-in-one-column-5.4"><span class="toc-item-num">5.4&nbsp;&nbsp;</span>Split all-in-one column</a></span><ul class="toc-item"><li><span><a href="#NIV" data-toc-modified-id="NIV-5.4.1"><span class="toc-item-num">5.4.1&nbsp;&nbsp;</span>NIV</a></span></li><li><span><a href="#IV" data-toc-modified-id="IV-5.4.2"><span class="toc-item-num">5.4.2&nbsp;&nbsp;</span>IV</a></span></li></ul></li></ul></li><li><span><a href="#Validation" data-toc-modified-id="Validation-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Validation</a></span><ul class="toc-item"><li><span><a href="#Data-type" data-toc-modified-id="Data-type-6.1"><span class="toc-item-num">6.1&nbsp;&nbsp;</span>Data type</a></span></li><li><span><a href="#Restoring-rows-when-data-got-mixed-with-headers" data-toc-modified-id="Restoring-rows-when-data-got-mixed-with-headers-6.2"><span class="toc-item-num">6.2&nbsp;&nbsp;</span>Restoring rows when data got mixed with headers</a></span><ul class="toc-item"><li><span><a href="#NIV" data-toc-modified-id="NIV-6.2.1"><span class="toc-item-num">6.2.1&nbsp;&nbsp;</span>NIV</a></span></li><li><span><a href="#IV" data-toc-modified-id="IV-6.2.2"><span class="toc-item-num">6.2.2&nbsp;&nbsp;</span>IV</a></span></li></ul></li><li><span><a href="#Concatenating" data-toc-modified-id="Concatenating-6.3"><span class="toc-item-num">6.3&nbsp;&nbsp;</span>Concatenating</a></span></li></ul></li><li><span><a href="#Appending-to-existing-data" data-toc-modified-id="Appending-to-existing-data-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>Appending to existing data</a></span></li><li><span><a href="#Removing-special-characters" data-toc-modified-id="Removing-special-characters-8"><span class="toc-item-num">8&nbsp;&nbsp;</span>Removing special characters</a></span></li><li><span><a href="#Output" data-toc-modified-id="Output-9"><span class="toc-item-num">9&nbsp;&nbsp;</span>Output</a></span><ul class="toc-item"><li><span><a href="#Local" data-toc-modified-id="Local-9.1"><span class="toc-item-num">9.1&nbsp;&nbsp;</span>Local</a></span></li><li><span><a href="#S3" data-toc-modified-id="S3-9.2"><span class="toc-item-num">9.2&nbsp;&nbsp;</span>S3</a></span></li></ul></li></ul></div>

# Objectives

- Built upon previous work [2023-06-30_cleaning_df_bothdata.py](https://github.com/tiangenglu/WebScrape/blob/main/06302023_cleaning_df_bothdata.py)
- Detect new raw data and ONLY process the newly scraped data
- After cleaning, append to the existing all-time visa data

In [1]:
import os
import pandas as pd
import boto3
import json
import io
import sys

In [2]:
%%time
old_data = pd.read_csv("visa_alltime.csv")

CPU times: user 132 ms, sys: 23 ms, total: 155 ms
Wall time: 155 ms


In [3]:
old_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 488523 entries, 0 to 488522
Data columns (total 5 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   nationality  488523 non-null  object
 1   visa         488523 non-null  object
 2   count        488523 non-null  int64 
 3   time         488523 non-null  object
 4   type         488523 non-null  object
dtypes: int64(1), object(4)
memory usage: 18.6+ MB


# Accessing scraped raw data in AWS S3

In [4]:
with open("aws_credential.txt", 'r') as file:
    aws_credential=json.load(file)
s3=boto3.Session(
    profile_name = None, 
    region_name = 'us-east-2').client(
    's3',
    aws_access_key_id=aws_credential['access_key'],
    aws_secret_access_key=aws_credential['secret_key'])

In [5]:
bucket_niv_content=s3.list_objects(Bucket = aws_credential['bucket'], Prefix ='messy_data/visa_scraped/niv/')['Contents']
bucket_iv_content=s3.list_objects(Bucket = aws_credential['bucket'], Prefix ='messy_data/visa_scraped/iv/')['Contents']

In [6]:
bucket_niv_items=[d['Key'].split('/')[-1] for d in bucket_niv_content]
bucket_iv_items=[d['Key'].split('/')[-1] for d in bucket_iv_content] 

In [7]:
bucket_iv_date=[f.split('.')[0].split('_')[-1] for f in bucket_iv_items]
bucket_niv_date=[f.split('.')[0].split('_')[-1] for f in bucket_niv_items]

# How many months (raw data) need processing? 

In [8]:
# better to use a list of unique time stamps
len_diff=len(bucket_niv_items) - len(old_data.time.unique())
if len_diff > 0:
    print(f'Need to append {len_diff} month(s) to existing NIV data.')
else: print("No actions required, compiled NIV data is up to date.")

Need to append 1 month(s) to existing NIV data.


In [9]:
# better to use a list of unique time stamps
len_diff=len(bucket_iv_items) - len(old_data.time.unique())
if len_diff > 0:
    print(f'Need to append {len_diff} month(s) to existing IV data.')
else: 
    print("No actions required, compiled IV data is up to date.")
    sys.exit(0) # exit(0) with grace

Need to append 1 month(s) to existing IV data.


In [10]:
old_data.time.max()

'2025-01-31'

In [11]:
max(bucket_niv_date)

'2025-02-28'

In [12]:
# last item minus the difference in length
bucket_niv_date[-len_diff-1] == old_data.time.max()

True

In [13]:
niv_date_to_add=bucket_niv_date[-len_diff:]
print(f'Needs the following month(s): {niv_date_to_add}')

Needs the following month(s): ['2025-02-28']


In [14]:
bucket_niv_date[-2:]

['2025-01-31', '2025-02-28']

In [15]:
iv_date_to_add = bucket_iv_date[-len_diff:]
print(f'Needs the following month(s): {iv_date_to_add} for IV.')

Needs the following month(s): ['2025-02-28'] for IV.


# Read in raw data from S3

## Non-immigrant data

In [16]:
niv_df_raw = [None] * len_diff

In [17]:
bucket_niv_items[-(0+1)]

'niv_2025-02-28.txt'

In [18]:
for i in range(len_diff):
    print(f'Getting the {-(i+1)} item from the .txt folder:')
    file = s3.get_object(Bucket = aws_credential['bucket'],
              # offset zero indexing: -(i+1), start from most recent (last in)
              Key = 'messy_data/visa_scraped/niv/'+bucket_niv_items[-(i+1)])['Body'].read()
    #niv_df_raw[i] = pd.DataFrame(file.decode("utf-8").split('\n')) # this works
    niv_df_raw[i] = pd.read_csv(io.BytesIO(file), delimiter = "\t", header = None) # also works

Getting the -1 item from the .txt folder:


In [19]:
test_object=s3.get_object(Bucket = aws_credential['bucket'], 
              Key = 'messy_data/visa_scraped/niv/' + bucket_niv_items[-len_diff])['Body'].read()

In [20]:
type(test_object)

bytes

In [21]:
test_object.decode("utf-8").split('\n')[:5]

['Nationality Visa Class Issuances',
 'Afghanistan A2 1',
 'Afghanistan B1/B2 127',
 'Afghanistan F1 18',
 'Afghanistan F2 1']

In [22]:
pd.DataFrame(test_object.decode("utf-8").split('\n')).iloc[:5]

,0
0,Nationality Visa Class Issuances
1,Afghanistan A2 1
2,Afghanistan B1/B2 127
3,Afghanistan F1 18
4,Afghanistan F2 1


In [23]:
pd.read_csv(io.BytesIO(test_object), delimiter = "\t", header = None).iloc[:5]

,0
0,Nationality Visa Class Issuances
1,Afghanistan A2 1
2,Afghanistan B1/B2 127
3,Afghanistan F1 18
4,Afghanistan F2 1


## Immigrant visa data

In [24]:
iv_df_raw = [None] * len_diff

In [25]:
for i in range(len_diff):
    print(f'Getting the {-(i+1)} item from the .txt folder:')
    file = s3.get_object(Bucket = aws_credential['bucket'],
              # offset zero indexing: -(i+1), start from most recent (last in)
              Key = 'messy_data/visa_scraped/iv/'+bucket_iv_items[-(i+1)])['Body'].read()
    #niv_df_raw[i] = pd.DataFrame(file.decode("utf-8").split('\n')) # this works
    iv_df_raw[i] = pd.read_csv(io.BytesIO(file), delimiter = "\t", header = None) # also works

Getting the -1 item from the .txt folder:


In [26]:
print(len(iv_df_raw))

1


# Cleaning

## A list of raw dataframes

In [27]:
grand_total = []
for i in range(len(niv_df_raw)):
    niv_df_raw[i].columns = ['V'] # more robust than .rename
    # or use .apply(lambda x: x.strip()), but the following is simple
    niv_df_raw[i]['V']=niv_df_raw[i]['V'].str.strip().str.upper()
    # insert an iterrows() loop to get the index of the grand total row
    for idx,row in niv_df_raw[i].iterrows():
        if 'grand total'.upper() in row['V']:
            grand_total.append(row)
            idx_rm_below = idx
    niv_df_raw[i]=niv_df_raw[i].iloc[:idx_rm_below]
    # offset zero indexing: -(i+1), start from most recent
    niv_df_raw[i]['time'] = niv_date_to_add[-(i+1)]
print(grand_total)

[V    GRAND TOTAL 914,989
Name: 3692, dtype: object]


In [28]:
iv_grand_total = []
for i in range(len(iv_df_raw)):
    iv_df_raw[i].columns = ['V']
    iv_df_raw[i]['V']=iv_df_raw[i]['V'].str.strip().str.upper()
    for idx,row in iv_df_raw[i].iterrows():
        if 'grand total'.upper() in row['V']:
            iv_grand_total.append(row)
            idx_rm_below = idx
    iv_df_raw[i] = iv_df_raw[i].iloc[:idx_rm_below]
    iv_df_raw[i]['time'] = iv_date_to_add[-(i+1)]
print(iv_grand_total)

[V    GRAND TOTAL 51,058
Name: 2010, dtype: object]


## Concatenated one long dataframe

In [29]:
df_raw_niv=pd.concat([df for df in niv_df_raw])
df_niv = df_raw_niv.copy(deep = True)

In [30]:
df_raw_iv=pd.concat([df for df in iv_df_raw])
df_iv=df_raw_iv.copy(deep=True)

## Remove non-data rows

### NIV

In [31]:
niv_headers = ['NONIMMIGRANT','NATIONALITY VISA','\\(FY', '\\#SBU','PAGE','SENSITIVE']
'|'.join([h for h in niv_headers])

'NONIMMIGRANT|NATIONALITY VISA|\\(FY|\\#SBU|PAGE|SENSITIVE'

In [32]:
# df_niv[df_niv['V'].str.len() <=1]
df_niv_headers=df_niv[df_niv['V'].str.contains('|'.join(niv_headers))]

In [33]:
df_niv_headers.index

Index([   0,   42,   43,   44,   45,   46,   89,   90,   91,  134,
       ...
       3556, 3599, 3600, 3601, 3644, 3645, 3646, 3689, 3690, 3691],
      dtype='int64', length=249)

In [34]:
# remove rows that were headers & footers, not final yet
df_niv=df_niv.iloc[~df_niv.index.isin(df_niv_headers.index)]

### IV

In [35]:
df_iv['V'] = df_iv['V'].str.strip()
df_iv = df_iv[df_iv['V'].str.len() > 1]
iv_headers = ['PAGE ', 'FOREIGN STATE OF', 'CHARGEABILITY', 
              'PLACE OF BIRTH', '\\(FY 20', '\\(FY20',
              'IMMIGRANT VISA', 'SENSITIVE']
df_iv_headers = df_iv.loc[df_iv['V'].str.contains('|'.join(iv_headers))]
df_iv = df_iv.iloc[~df_iv.index.isin(df_iv_headers.index)]

In [36]:
df_iv_headers.shape

(270, 2)

In [37]:
df_iv.shape

(1740, 2)

## Split all-in-one column

### NIV

In [38]:
df_niv.head(2)

,V,time
1,AFGHANISTAN A2 1,2025-02-28
2,AFGHANISTAN B1/B2 127,2025-02-28


In [39]:
# get rid of the warning messages
pd.options.mode.copy_on_write = True
df_niv['nationality']=[' '.join(row.split(' ')[:-2]).strip() for row in df_niv['V']]
# visa class
df_niv['visa']=[row.split(' ')[-2].strip() for row in df_niv['V']]
# remove thousand separator , from numbers
df_niv['issue']=[row.split(' ')[-1].replace(',','').strip() for row in df_niv['V']]

In [40]:
df_niv.head(2)

,V,time,nationality,visa,issue
1,AFGHANISTAN A2 1,2025-02-28,AFGHANISTAN,A2,1
2,AFGHANISTAN B1/B2 127,2025-02-28,AFGHANISTAN,B1/B2,127


### IV

In [41]:
df_iv['nationality']=[' '.join(row.split(' ')[:-2]).strip() for row in df_iv['V']]
df_iv['visa']=[row.split(' ')[-2].strip() for row in df_iv['V']]
df_iv['issue']=[row.split(' ')[-1].replace(',','').strip() for row in df_iv['V']]

In [42]:
df_iv.head(2)

,V,time,nationality,visa,issue
2,AFGHANISTAN CR1 25,2025-02-28,AFGHANISTAN,CR1,25
3,AFGHANISTAN DV 109,2025-02-28,AFGHANISTAN,DV,109


# Validation

## Data type

In [43]:
# test list comprehension with a short and simple list
[s for s in ['34',23,'a1','20','b '] if not str(s).isdigit()]

['a1', 'b ']

In [44]:
# Are there non-numeric values in the visa issuance count column?
check_numeric=[s for s in df_niv['issue'] if not str(s).isdigit()]
if len(check_numeric)>0:
    print("At least one row has non-numeric values in the NIV issuance column. Go back and check.")
    print(check_numeric)
    sys.exit()
else:
    print("No non-numeric values were detected in the NIV issuance column. Good to proceed.")
    df_niv['issue'] = df_niv['issue'].astype(int)

No non-numeric values were detected in the NIV issuance column. Good to proceed.


In [45]:
# Are there non-numeric values in the visa issuance count column?
check_numeric=[s for s in df_iv['issue'] if not str(s).isdigit()]
if len(check_numeric)>0:
    print("At least one row has non-numeric values in the IV issuance column. Go back and check.")
    print(check_numeric)
    sys.exit()
else:
    print("No non-numeric values were detected in the IV issuance column. Good to proceed.")
    df_iv['issue'] = df_iv['issue'].astype(int)

No non-numeric values were detected in the IV issuance column. Good to proceed.


## Restoring rows when data got mixed with headers

- This is the most challenging part of cleaning this dataset.
- `if any(pattern in input_text for pattern in pattern_list):`

### NIV

In [46]:
restore_idx = []
# here's how any() works
for idx,row in df_niv_headers.iterrows():
    if any(c in row['V'] for c in df_niv.nationality.unique()):
        print(idx, row)
        restore_idx.append(idx)

42 V       ALGERIA G2 11NONIMMIGRANT VISA ISSUANCES BY NA...
time                                           2025-02-28
Name: 42, dtype: object


In [47]:
df_restore=df_niv_headers.loc[restore_idx]

In [48]:
df_restore

,V,time
42,ALGERIA G2 11NONIMMIGRANT VISA ISSUANCES BY NA...,2025-02-28


In [49]:
df_restore['nationality']=[' '.join(row.split('NONIMMIGRANT')[0].strip().split(' ')[:-2]).strip() for row in df_restore['V']]
df_restore['visa'] = [row.split('NONIMMIGRANT')[0].strip().split(' ')[-2].strip() for row in df_restore['V']]
df_restore['issue'] = [row.split('NONIMMIGRANT')[0].strip().split(' ')[-1].strip().replace(',','') for row in df_restore['V']]
df_restore['issue'] = df_restore['issue'].astype(int)

In [50]:
df_restore

,V,time,nationality,visa,issue
42,ALGERIA G2 11NONIMMIGRANT VISA ISSUANCES BY NA...,2025-02-28,ALGERIA,G2,11


### IV

In [51]:
restore_idx_iv = []
# here's how any() works
for idx,row in df_iv_headers.iterrows():
    if any(c in row['V'] for c in df_iv.nationality.unique()):
        print(idx, row)
        restore_idx_iv.append(idx)

40 V       ANGOLA IR5 3                           IMMIGRA...
time                                           2025-02-28
Name: 40, dtype: object


In [52]:
df_restore_iv=df_iv_headers.loc[restore_idx_iv]
df_restore_iv['nationality']=[' '.join(row.split('IMMIGRANT')[0].strip().split(' ')[:-2]).strip() for row in df_restore_iv['V']]
df_restore_iv['visa'] = [row.split('IMMIGRANT')[0].strip().split(' ')[-2].strip() for row in df_restore_iv['V']]
df_restore_iv['issue'] = [row.split('IMMIGRANT')[0].strip().split(' ')[-1].strip().replace(',','') for row in df_restore_iv['V']]
df_restore_iv['issue'] = df_restore_iv['issue'].astype(int)
df_restore_iv

,V,time,nationality,visa,issue
40,ANGOLA IR5 3 IMMIGRA...,2025-02-28,ANGOLA,IR5,3


## Concatenating

In [53]:
col_order = ['nationality', 'visa', 'issue','time']

In [54]:
df_niv=pd.concat([df_niv, df_restore]).sort_index().drop(columns = ['V'])[col_order].drop_duplicates()

In [55]:
pd.DataFrame(grand_total)

,V
3692,"GRAND TOTAL 914,989"


In [56]:
df_niv['issue'].sum()

914989

In [57]:
df_iv=pd.concat([df_iv, df_restore_iv]).sort_index().drop(columns=['V'])[col_order].drop_duplicates()

In [58]:
df_iv['issue'].sum()

51058

In [59]:
df_niv = df_niv.rename(columns={'issue':'count'})
df_iv = df_iv.rename(columns={'issue':'count'})

# Appending to existing data

In [60]:
niv_alltime=s3.get_object(Bucket = aws_credential['bucket'], 
              Key = 'visa_output/niv_alltime.csv')['Body'].read()
df_niv_alltime = pd.read_csv(io.BytesIO(niv_alltime),low_memory=True)

In [61]:
iv_alltime=s3.get_object(Bucket = aws_credential['bucket'], 
              Key = 'visa_output/iv_alltime.csv')['Body'].read()
df_iv_alltime = pd.read_csv(io.BytesIO(iv_alltime),low_memory=True)

In [62]:
df_niv_alltime.shape

(317965, 4)

In [63]:
df_iv_alltime.shape

(170558, 4)

In [64]:
df_niv_alltime_new=pd.concat([df_niv_alltime, df_niv]).reset_index(drop=True).drop_duplicates()

In [65]:
df_iv_alltime_new=pd.concat([df_iv_alltime, df_iv]).reset_index(drop=True).drop_duplicates()

# Removing special characters

In [66]:
country_list=list(set(list(df_iv_alltime_new['nationality'].unique()) + 
                      list(df_niv_alltime_new['nationality'].unique()
                          )
                     )
                 )
print("Total unique country/nationality labels before cleaning: ",len(country_list))

Total unique country/nationality labels before cleaning:  270


In [67]:
special_chars = []
# instead of iterrows, can also work on a list of unique nationalities
for country in country_list:
    for char in country:
        if not (char.isalpha() or char == ' '):
            if char not in special_chars:
                special_chars.append(char)
                print(char, country)

' COTE D'IVOIRE
- HONG KONG-BNO
, CONGO, DEMOCRATIC REPUBLIC OF THE
. MACAU S. A. R.
* ESWATINI*
& GREAT BRITAIN & NORTHERN IRELAND
( NORTHERN IRELAND (DV ONLY)
) NORTHERN IRELAND (DV ONLY)
– CHINA – MAINLAND BORN


In [68]:
special_chars.remove(',') # potential legit
special_chars.remove("'") # potential legit
special_chars

['-', '.', '*', '&', '(', ')', '–']

In [69]:
old_country_label = []
new_country_label = []
for country in country_list:
    # the following covers the case when one string contains multiple special characters: e.g., '(' and ')'
    if any(char in country for char in special_chars):
        # if it's a letter or a space, join as usual, then replace special character with a space
        new_country = ''.join([char if (char.isalpha() or char == ' ') 
                               else char.replace(char,' ') 
                               for char in country]) # replace special character with space
        new_country = ' '.join(new_country.split()).replace('BORN','').strip() # split() to remove excessive space
        old_country_label.append(country)
        new_country_label.append(new_country)
        print("\nold: ",country,'\nnew: ', new_country)


old:  HONG KONG-BNO 
new:  HONG KONG BNO

old:  CHINA - TAIWAN BORN 
new:  CHINA TAIWAN

old:  MACAU S. A. R. 
new:  MACAU S A R

old:  ESWATINI* 
new:  ESWATINI

old:  GUINEA-BISSAU 
new:  GUINEA BISSAU

old:  CHINA-TAIWAN 
new:  CHINA TAIWAN

old:  *NON-NATIONALITY BASED ISSUANCES 
new:  NON NATIONALITY BASED ISSUANCES

old:  ST. VINCENT AND THE GRENADINES 
new:  ST VINCENT AND THE GRENADINES

old:  GREAT BRITAIN & NORTHERN IRELAND 
new:  GREAT BRITAIN NORTHERN IRELAND

old:  CHINA-MAINLAND BORN 
new:  CHINA MAINLAND

old:  NORTHERN IRELAND (DV ONLY) 
new:  NORTHERN IRELAND DV ONLY

old:  HONG KONG S.A.R 
new:  HONG KONG S A R

old:  ST. PIERRE AND MIQUELON 
new:  ST PIERRE AND MIQUELON

old:  ST. HELENA 
new:  ST HELENA

old:  CHINA-TAIWAN BORN 
new:  CHINA TAIWAN

old:  COCOS (KEELING) ISLANDS 
new:  COCOS KEELING ISLANDS

old:  NON-NATIONALITY BASED ISSUANCES 
new:  NON NATIONALITY BASED ISSUANCES

old:  HONG KONG S.A.R. 
new:  HONG KONG S A R

old:  BURKINA-FASO 
new:  BURKINA F

In [70]:
no_sp_char_label=dict(zip(old_country_label,new_country_label))

In [71]:
# map new country labels to a new column nationality2, then replace it with original nationality
df_niv_alltime_new['nationality2'] = df_niv_alltime_new['nationality'].map(
    no_sp_char_label).fillna(
    df_niv_alltime_new['nationality'])
df_iv_alltime_new['nationality2'] = df_iv_alltime_new['nationality'].map(
    no_sp_char_label).fillna(
    df_iv_alltime_new['nationality'])
df_niv_alltime_new = df_niv_alltime_new.drop(
    columns=['nationality']).rename(
    columns={'nationality2':'nationality'})
df_iv_alltime_new = df_iv_alltime_new.drop(
    columns=['nationality']).rename(
    columns={'nationality2':'nationality'})

In [72]:
country_list_new=list(set(list(df_iv_alltime_new['nationality'].unique()) + 
                      list(df_niv_alltime_new['nationality'].unique()
                          )
                     )
                 )
print("Total unique country/nationality labels after cleaning: ",len(country_list_new))
print(f'After removing special characters, {len(country_list) - len(country_list_new)} labels were reduced.')

Total unique country/nationality labels after cleaning:  258
After removing special characters, 12 labels were reduced.


# Output

## Local

In [73]:
with open('country_list.txt','w') as file:
    file.write('\n'.join(country_list_new))

In [74]:
new_col_order = ['nationality','visa', 'count', 'time']
df_niv_alltime_new = df_niv_alltime_new[new_col_order]
df_iv_alltime_new = df_iv_alltime_new[new_col_order]

In [75]:
time_niv=list(df_niv_alltime_new['time'].unique())
time_iv=list(df_iv_alltime_new['time'].unique())

In [76]:
# output a list of time stamps
with open('time_niv.txt', 'w') as file:
    file.write('\n'.join(time_niv))
with open('time_iv.txt', 'w') as file:
    file.write('\n'.join(time_iv))

## S3

In [77]:
# make list a string to upload
s3.put_object(Body = "\n".join([c for c in country_list_new]), 
              Bucket = aws_credential['bucket'], 
              Key = 'visa_output/country_list.txt')
s3.put_object(Body = "\n".join([t for t in time_niv]), 
              Bucket = aws_credential['bucket'], 
              Key = 'visa_output/time_niv.txt')
s3.put_object(Body = "\n".join([t for t in time_iv]), 
              Bucket = aws_credential['bucket'], 
              Key = 'visa_output/time_iv.txt')
# list objects after upload
output_folder_items=s3.list_objects(Bucket = aws_credential['bucket'], Prefix = 'visa_output')['Contents']
item_names=[d['Key'] for d in output_folder_items]
[item for item in item_names if item.endswith('.txt')]

['visa_output/country_list.txt',
 'visa_output/time_iv.txt',
 'visa_output/time_niv.txt']

In [78]:
# upload pandas dataframe to s3
csv_buffer = io.StringIO()
df_niv_alltime_new.to_csv(csv_buffer, index=False)
s3.put_object(Body = csv_buffer.getvalue(), 
              Bucket = aws_credential['bucket'], 
              Key = 'visa_output/df_niv.csv')
# create a new csv buffer object to upload a different data frame
csv_buffer = io.StringIO()
df_iv_alltime_new.to_csv(csv_buffer, index=False)
s3.put_object(Body = csv_buffer.getvalue(), 
              Bucket = aws_credential['bucket'], 
              Key = 'visa_output/df_iv.csv')
# list objects after upload
output_folder_items=s3.list_objects(Bucket = aws_credential['bucket'], Prefix = 'visa_output')['Contents']
item_names=[d['Key'] for d in output_folder_items]
[item for item in item_names if item.endswith('.csv')]

['visa_output/df_iv.csv',
 'visa_output/df_niv.csv',
 'visa_output/iv_alltime.csv',
 'visa_output/niv_alltime.csv']